In [1]:
from dotenv import load_dotenv
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

load_dotenv()

/Users/jinto/.local/share/virtualenvs/langchain-0u9w6_h2/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


True

## Set API Key if required

In [8]:
# Verify API key is loaded
api_key = os.getenv("OPENAI_API_KEY")
print(f"API Key loaded: {bool(api_key)}")

API Key loaded: True


## Information to Analyze

In [9]:
info = """
Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter, and xAI. Musk has been the wealthiest person in the world since 2025; as of February 2026, Forbes estimates his net worth to be around US$852 billion.
Born into a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he has Canadian citizenship since his mother was born there. He received bachelor's degrees in 1997 from the University of Pennsylvania before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. Musk also became an American citizen in 2002.
In 2002, Musk founded the space technology company SpaceX, becoming its CEO and chief engineer; the company has since led innovations in reusable rockets and commercial spaceflight. Musk joined the automaker Tesla as an early investor in 2004 and became its CEO and product architect in 2008; it has since become a leader in electric vehicles. In 2015, he co-founded OpenAI to advance artificial intelligence (AI) research, but later left; growing discontent with the organization's direction and their leadership in the AI boom in the 2020s led him to establish xAI, which became a subsidiary of SpaceX in 2026. In 2022, he acquired the social network Twitter, implementing significant changes, and rebranding it as X in 2023. His other businesses include the neurotechnology company Neuralink, which he co-founded in 2016, and the tunneling company the Boring Company, which he founded in 2017. In November 2025, a Tesla pay package worth $1 trillion for Musk was approved, which he is to receive over 10 years if he meets specific goals.
Musk was the largest donor in the 2024 U.S. presidential election, where he supported Donald Trump. After Trump was inaugurated as president in early 2025, Musk served as Senior Advisor to the President and as the de facto head of the Department of Government Efficiency (DOGE). After a public feud with Trump, Musk left the Trump administration and returned to managing his companies. Musk is a supporter of global far-right figures, causes, and political parties. His political activities, views, and statements have made him a polarizing figure. Musk has been criticized for COVID-19 misinformation, promoting conspiracy theories, and affirming antisemitic, racist, and transphobic comments. His acquisition of Twitter was controversial due to a subsequent increase in hate speech and the spread of misinformation on the service, following his pledge to decrease censorship. His role in the second Trump administration attracted public backlash, particularly in response to DOGE. The emails he sent to Jeffrey Epstein are included in the Epstein files, which were published between 2025–26 and became a topic of worldwide debate.
"""

print(f"Information length: {len(info)} characters")

Information length: 2990 characters


## Set Up Prompt Template

In [10]:
template = """
Given the information {information} of a person, you are going to return 
1. A summary of the person in 3 sentences.
2. 2 interesting facts about the person.
"""

prompt = PromptTemplate(
    input_variables=["information"],
    template=template
)

print("Prompt template created successfully")

Prompt template created successfully


## Create LLM Chain and Get Response

In [11]:
# Initialize the LLM with specific model and temperature
# Lower temperature for more deterministic output, higher for more creative
llm = ChatOllama(model="gpt-oss:20b", temperature=0.3)

# Create a runnable chain connecting the prompt and the LLM
chain = prompt | llm

print("Chain created successfully")

Chain created successfully


In [12]:
# Invoke the chain with the information
response = chain.invoke(input={"information": info})
print(response)

content='**Summary (3 sentences)**  \nElon Musk, born in 1971 in Pretoria, is a South African‑born entrepreneur who has built a global empire spanning electric vehicles, spaceflight, AI, and social media. After co‑founding Zip2, X.com (which became PayPal), and later SpaceX, he became CEO of Tesla and acquired Twitter, rebranding it as X in 2023. In 2025 he was the world’s richest person, and his political and public statements—ranging from far‑right endorsements to controversial social media policies—have made him a polarizing figure worldwide.  \n\n**Interesting facts**  \n1. Musk’s 2025 Tesla pay package, approved by shareholders, is valued at a staggering $1\u202ftrillion and will be paid over a decade if performance targets are met.  \n2. In 2026, his AI venture xAI became a subsidiary of SpaceX, marking the first time an AI company was formally integrated into a space‑flight organization.' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-24T1

## Agents

In [6]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama


### Custom Tool

In [10]:
from tavily import TavilyClient
tavily_tool=TavilyClient()
@tool
def custom_search(query: str) -> str:
    """
    Tool that searches internet
    Parameters:
        query (str): The search query to be executed.
    Returns:
        str: The search result.
    """
    return tavily_tool.search(query)
tools = [custom_search]
agent = create_agent(
    model=ChatOllama(model="gpt-oss:20b", temperature=0.3),
    tools=tools
)
result = agent.invoke({"messages":HumanMessage(content="What is the current stock price of Tesla?")})
print(result)

{'messages': [HumanMessage(content='What is the current stock price of Tesla?', additional_kwargs={}, response_metadata={}, id='1611f29e-8255-4b38-86a3-c5ed2ab051ec'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-24T15:14:38.750312Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1662139000, 'load_duration': 109010041, 'prompt_eval_count': 151, 'prompt_eval_duration': 436483083, 'eval_count': 52, 'eval_duration': 1104959412, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9037-6b9f-71d1-8c19-3038ec99cba7-0', tool_calls=[{'name': 'custom_search', 'args': {'query': 'Tesla current stock price'}, 'id': 'c8bf9875-f70b-4017-b20d-d16460f3f5eb', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 151, 'output_tokens': 52, 'total_tokens': 203}), ToolMessage(content='{"query": "Tesla current stock price", "follow_up_questions": null, "answer": null, "ima

### Inbuilt Tool

In [12]:
from langchain_tavily import TavilySearch

agent = create_agent(
    model=ChatOllama(model="gpt-oss:20b", temperature=0.3),
    tools=[TavilySearch()]
)
result = agent.invoke({"messages":HumanMessage(content="What is the current stock price of Tesla?")})
print(result)

{'messages': [HumanMessage(content='What is the current stock price of Tesla?', additional_kwargs={}, response_metadata={}, id='f5ae2d3e-5650-4a1b-9cc4-c09aaccce292'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2026-02-24T15:15:57.739229Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5586171917, 'load_duration': 114497333, 'prompt_eval_count': 1254, 'prompt_eval_duration': 2832021042, 'eval_count': 97, 'eval_duration': 2615947421, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--019c9038-90d7-7523-be4e-8a28c3136e14-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Tesla stock price', 'search_depth': 'fast', 'include_images': False}, 'id': '2d8c2d49-36c6-4754-bb7f-183739ce8871', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1254, 'output_tokens': 97, 'total_tokens': 1351}), ToolMessage(content='{"query": "Tesla stock price", "follow_up_q

### Structured Response

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Sources(BaseModel):
    """Schema for sources in the agent response."""
    url:str = Field(description="The URL of the source")
class AgentResponse(BaseModel):
    """Schema for the agent response."""
    answer: str = Field(description="The answer generated by the agent")
    sources: List[Sources] = Field(description="List of sources used by the agent to generate the answer")

# by default langchain used ProviderStratergy and if ollama or model doesn't support it, it will throw an error. 
# Client.chat() got an unexpected keyword argument 'response_format'
# So we need to set the response format to our custom schema
agent = create_agent(
    model=ChatOllama(model="gpt-oss:20b", temperature=0.3),
    tools=[TavilySearch()],
    response_format=AgentResponse
)
result = agent.invoke({"messages":HumanMessage(content="What is the current stock price of Tesla?")})
print(result)

TypeError: Client.chat() got an unexpected keyword argument 'response_format'